In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
premise_file = "../Resources/LicensePremise.csv"


In [3]:
county_file = "../Resources/CountyLicenseCount.csv"


### Transform premise DataFrame

In [36]:
license=pd.read_csv(premise_file)
countydf=pd.read_csv(county_file)
license.head()
countydf.head()


,ID,County Name (Licensee),County ID Code,License Count
0,0,ALBANY,5,77
1,1,ALLEGANY,59,4
2,2,BRONX,0,104
3,3,BROOME,35,14
4,4,CATTARAUGUS,41,9


### Transform county DataFrame

In [37]:
# A `premise` table that contains the columns `id`, `premise_name` and `county_id`.

# A `county` table that contains the columns `id`, `county_name`, `license_count` and `county_id`.
premise=license[["License Serial Number", "Premises Name", "County ID Code"]].copy()
county=countydf[["ID", "County Name (Licensee)", "License Count", "County ID Code"]]
county=county.rename(columns={"County Name (Licensee)":"county_name"
                             ,"County ID Code":"county_id"
                    ,"License Count":"license_count"
                    ,"ID":"id"
                             })

premise=premise.rename(columns={"License Serial Number":"id", "Premises Name":"premise_name", "County ID Code":"county_id"
                             })

premise.head()

,id,premise_name,county_id
0,1311660,CANA ARRIBA GROCERY NO 2 INC,0
1,1310023,JOHANA GROCERY & DELI CORP,0
2,1310024,TKO BEVERAGES LLC,1
3,1310024,TKO BEVERAGES LLC,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,2


### Create database connection

In [38]:
rds_connection_string = "postgres:Nnms@1196@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [39]:
# Confirm tables
engine.table_names()

['customer_name', 'customer_location']

### Load DataFrames into database

In [40]:
premise.to_sql(name='premise', con=engine, if_exists='append', index=False)
county.to_sql(name='county', con=engine, if_exists='append', index=False)

In [41]:
pd.read_sql_query('select * from premise', con=engine).head()

,id,premise_name,county_id
0,1311660,CANA ARRIBA GROCERY NO 2 INC,0
1,1310023,JOHANA GROCERY & DELI CORP,0
2,1310024,TKO BEVERAGES LLC,1
3,1310024,TKO BEVERAGES LLC,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,2


In [42]:
pd.read_sql_query('select * from county', con=engine).head()

,id,county_name,license_count,county_id
0,0,ALBANY,77,5
1,1,ALLEGANY,4,59
2,2,BRONX,104,0
3,3,BROOME,14,35
4,4,CATTARAUGUS,9,41
